In [ ]:
!pip install datasets pinecone-client sentence_transformers torch-scatter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
retriever = SentenceTransformer("deepset/all-mpnet-base-v2-table", device=device)
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [ ]:
import pandas as pd
tables=[]
table = pd.read_excel("/content/CRTD_BUDGET_AOP.xlsx")
table = table.astype(str)
tables.append(table)
table = pd.read_excel("/content/CRTD_ITEM_LOCATION.xlsx")
table = table.astype(str)
tables.append(table)
table = pd.read_excel("/content/CRTD_DAILY_DMDPROMO_VIEW.xlsx")
table = table.astype(str)
tables.append(table)

In [ ]:
def _preprocess_tables(tables: list):
    processed = []
    for table in tables:
        processed_table = "\n".join([table.to_csv(index=False)])
        processed.append(processed_table)
    return processed

In [ ]:
processed_tables = _preprocess_tables(tables)

In [ ]:
import pinecone
pinecone.init(
    api_key="58857173-64a0-4cc2-b90c-599efcf3c693",
    environment="gcp-starter"
)

In [ ]:
index_name = "tumpa-sona-duto-hampi-dena"
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=768,
        metric="cosine"
    )
index = pinecone.Index(index_name)

In [ ]:
from tqdm.auto import tqdm

batch_size = 3

for i in tqdm(range(0, len(processed_tables), batch_size)):
    i_end = min(i+batch_size, len(processed_tables))
    batch = processed_tables[i:i_end]
    emb = retriever.encode(batch).tolist()
    ids = [f"{idx}" for idx in range(i, i_end)]
    to_upsert = list(zip(ids, emb))
    _ = index.upsert(vectors=to_upsert)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from transformers import pipeline, TapasTokenizer, TapasForQuestionAnswering

model_name = "google/tapas-base-finetuned-wtq"
tokenizer = TapasTokenizer.from_pretrained(model_name)
model = TapasForQuestionAnswering.from_pretrained(model_name, local_files_only=False)
pipe = pipeline("table-question-answering",  model=model, tokenizer=tokenizer, device=device)

In [ ]:
def query_pinecone(query):
    xq = retriever.encode([query]).tolist()
    result = index.query(xq, top_k=1)
    return tables[int(result["matches"][0]["id"])]

In [ ]:
def get_answer_from_table(table, query):
    answers = pipe(table=table, query=query)
    return answers

In [ ]:
query = "What is the highest budget in Canada through ONLINE-ECOM?"
table = query_pinecone(query)
get_answer_from_table(table, query)

{'answer': 'AVERAGE > 6850883.0',
 'coordinates': [(6, 3)],
 'cells': ['6850883.0'],
 'aggregator': 'AVERAGE'}

In [ ]:
query = "What are the locations where body care products are sold?"
table = query_pinecone(query)
print(table)
get_answer_from_table(table, query)

          PRODUCTID      CUSTOMER LOCATION BUDGETAMOUNT STARTDATE     DUR  \
0         Body Care   ONLINE_ECOM   Canada    5240926.0  00:00:00  525600   
1         Body Care   ONLINE_ECOM   Canada      47942.7  00:00:00  525600   
2         Body Care   ONLINE_ECOM   Canada     87543.45  00:00:00  525600   
3         Body Care   ONLINE_ECOM   Canada    6165795.0  00:00:00  525600   
4         Body Care   ONLINE_ECOM   Canada  53269.66667  00:00:00  525600   
..              ...           ...      ...          ...       ...     ...   
186  Face Cleansing  ALL CUSTOMER  ALL LOC      32528.0  00:00:00   43200   
187  Face Cleansing  ALL CUSTOMER  ALL LOC      31946.0  00:00:00   44640   
188  Face Cleansing  ALL CUSTOMER  ALL LOC      32911.0  00:00:00   43200   
189  Face Cleansing  ALL CUSTOMER  ALL LOC      33408.0  00:00:00   43200   
190             PL4     HYPERMART       L2     100000.0  00:00:00  525600   

    INPUTID ACTIONCODE                             MESSAGEID  \
0    MARKET

{'answer': 'Canada',
 'coordinates': [(0, 2)],
 'cells': ['Canada'],
 'aggregator': 'NONE'}

In [ ]:
query = "What is the mathematical sum of budgets in Canada through ONLINE-ECOM?"
table = query_pinecone(query)
get_answer_from_table(table, query)

{'answer': 'SUM > 5240926.0, 47942.7, 87543.45, 6165795.0, 53269.66667, 97270.5, 6850883.0, 56073.33333, 102390.0',
 'coordinates': [(0, 3),
  (1, 3),
  (2, 3),
  (3, 3),
  (4, 3),
  (5, 3),
  (6, 3),
  (7, 3),
  (8, 3)],
 'cells': ['5240926.0',
  '47942.7',
  '87543.45',
  '6165795.0',
  '53269.66667',
  '97270.5',
  '6850883.0',
  '56073.33333',
  '102390.0'],
 'aggregator': 'SUM'}